In [526]:
# Install packages as needed
#%pip install openpyxl
#%pip install pandas 
#%pip install plotly
#%pip install nbformat
#%pip install --upgrade ipython

In [527]:
#import libraries
import pandas as pd
import json
import os
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
import locale
import numpy as np
import re

Extract

In [528]:
###EXTRACT###
# Read Transaction Excel data into a Pandas dataframe
file = r'C:\Users\DIM6\OneDrive - PGE\[Work] [Shared]\CXI Projects\20200122 03 CC Fact Sheet\2022\2022 Customer Transactions Map - Publication Sheets.xlsx'
sheet_name = 'Sunburst'

# Read the specific sheet from the Excel file
df_raw = pd.read_excel(file, sheet_name=sheet_name)
print(df_raw.head())

  Root Level Level 1 Level 2 L3 Classifier  Value  TableBlank?  Level  \
0        NaN     NaN     NaN           NaN    NaN        False    NaN   
1        NaN     NaN     NaN           NaN    NaN        False    NaN   
2        NaN     NaN     NaN           NaN    NaN        False    NaN   
3        NaN     NaN     NaN           NaN    NaN        False    NaN   
4        NaN     NaN     NaN           NaN    NaN        False    NaN   

   Unnamed: 7                        0 0.1 59.844395  
0         NaN   % Payment transactions   0      2022  
1         NaN              Pay by Mail   0  0.187152  
2         NaN                      Web   0  0.467045  
3         NaN  Other Elec Pay Channels   0  0.243905  
4         NaN              Call Center   0  0.072321  


In [529]:
 #Remove columns not needed for sunburst
df = df[['Root Level', 'Level 1', 'Level 2', 'Value']]

# convert cells in column "Value" to numeric values
#df['Value'] = pd.to_numeric(df['Value'], errors='coerce')

# Remove rows with all NaN values
#filtered_df = df.dropna(how='all')

# filter out rows with None, blank, or NaN values in the "Value" column
#df = filtered_df.loc[pd.isna(filtered_df['Value']) | (filtered_df['Value'] == '') | pd.notna(filtered_df['Value'])]

print(df.head())

          Root Level   Level 1 Level 2   Value
43               NPC  Payments     Nan   1.770
49              Mail  Payments     Nan  11.200
58  Other Electronic  Payments     Nan  10.393
59  Other Electronic  Payments     Nan   2.735
60  Other Electronic  Payments     Nan   0.637


In [530]:
# Remove "-" and convert to proper case
df["Root Level"] = df["Root Level"].str.replace(r"\s*-\s*", "").astype(str)
df["Level 1"] = df["Level 1"].str.replace(r"\s*-\s*", "").astype(str)
df["Level 2"] = df["Level 2"].str.replace(r"\s*-\s*", "").astype(str)

#Convert L2 to proper case
df["Level 2"] = df["Level 2"].str.title()

C:\Users\DIM6\AppData\Local\Temp\1\ipykernel_14416\2126742580.py:2: FutureWarning:

The default value of regex will change from True to False in a future version.

C:\Users\DIM6\AppData\Local\Temp\1\ipykernel_14416\2126742580.py:3: FutureWarning:

The default value of regex will change from True to False in a future version.

C:\Users\DIM6\AppData\Local\Temp\1\ipykernel_14416\2126742580.py:4: FutureWarning:

The default value of regex will change from True to False in a future version.



In [531]:
# Splice a new df
df = df.copy()

print(df.head())

          Root Level   Level 1 Level 2   Value
43               NPC  Payments     Nan   1.770
49              Mail  Payments     Nan  11.200
58  Other Electronic  Payments     Nan  10.393
59  Other Electronic  Payments     Nan   2.735
60  Other Electronic  Payments     Nan   0.637


In [532]:
# Convert dataframe to a hierarchical dictionary
def make_hierarchy_dict(df, path=[], value_col="Value", threshold=10):
    if len(path) == df.columns.nlevels - 1:
        total = int(df[value_col].sum())
        if total < threshold:
            return None
        else:
            return total
    else:
        level = df.columns[len(path)]
        subdict = {}
        for key in df[level].dropna().unique():
            subdf = df[df[level] == key].drop(columns=[level])
            subhierarchy_dict = make_hierarchy_dict(subdf, path + [key], value_col, threshold)
            if subhierarchy_dict is not None:
                subdict[key] = subhierarchy_dict
        if len(subdict) == 0:
            return None
        else:
            return subdict

hierarchy_dict = make_hierarchy_dict(df.set_index(list(df.columns[:3])), value_col="Value", threshold=10)

print(hierarchy_dict)

177


In [533]:
# Convert hierarchical dictionary to Plotly-compatible JSON
def make_plotly_sunburst(hierarchy_dict, name=""):
    if isinstance(hierarchy_dict, dict):
        children = []
        for key, value in hierarchy_dict.items():
            children.append(make_plotly_sunburst(value, key))
        return {
            "name": name,
            "children": children
        }
    else:
        return {
            "label": name,
            "value": hierarchy_dict
        }

In [534]:
plotly_data = make_plotly_sunburst(hierarchy_dict)
print(plotly_data)

{'label': '', 'value': 177}


In [535]:
# Write JSON data to a file
with open("plotly_data.json", "w") as f:
    json.dump(plotly_data, f)

In [536]:
# Write dataframe to an Excel file
with pd.ExcelWriter('dataframe copy.xlsx') as writer:
    df.to_excel(writer, sheet_name='Sheet1', index=False)

In [537]:
 #Create a dictionary to map the Root Level to its corresponding color
color_map = {'Web': 'orange', 'Other Electronic': 'gray', 'Mail': 'burlywood', 'Call Center - IVR': 'deepskyblue','Call Center - Other':'lightblue', \
              'Call Center - Live Agent':'Turquoise', 'NPC': 'gold'}

# Format a sunburst chart using Plotly Express
color_sequence = [color_map[key] for key in color_map]
fig = px.sunburst(df, path=["Root Level", "Level 1", "Level 2"], values='Value')

# Update the layout
fig.update_layout(
    title='Transaction Channel Volume 2022',
    font=dict(size=14),
    margin=dict(t=50, l=0, r=0, b=0),
    width=1400,
    height=900,
    sunburstcolorway=color_sequence,
)
fig.show()


In [538]:
# Export the chart as an HTML file
#pyo.plot(fig, filename='Trxn_sunburst_chart.html', auto_open=True)